In [1]:
!python -V

Python 3.12.3


In [2]:
import math
import os.path
import re
import sys
from os import path

from loguru import logger

from app.config import config
from app.models import const
from app.models.schema import VideoParams, VideoConcatMode
from app.services import llm, material, voice, video, subtitle
from app.services import state as sm
from app.utils import utils

2024-05-13 22:43:47.803 | INFO     | app.config.config:load_config:22 - load config from file: /Users/roper/work/auto_video_generator/config.toml
2024-05-13 22:43:47.806 | INFO     | app.config.config:<module>:70 - MoneyPrinterTurbo v1.1.5
/Users/roper/miniconda3/envs/auto-upload/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
params_dict = {
        "video_subject": "怎么快速赚到100万",
        "video_aspect": "9:16",
        "voice_name": "zh-CN-XiaoxiaoNeural",
        "enable_bgm": False,
        "font_name": "STHeitiMedium 黑体-中",
        "text_color": "#FFFFFF",
        "font_size": 60,
        "stroke_color": "#000000",
        "stroke_width": 1.5
    }

params = VideoParams(**params_dict)
task_id = 'task_id_test_123'

In [4]:
params.video_subject


'怎么快速赚到100万'

In [5]:
config.app.get("openai_model_name", "openai")

'gpt-3.5-turbo'

In [6]:
video_subject = params.video_subject
voice_name = voice.parse_voice_name(params.voice_name)
paragraph_number = params.paragraph_number
n_threads = params.n_threads
max_clip_duration = params.video_clip_duration
logger.info("\n\n## generating video script")
video_script = params.video_script.strip()
if not video_script:
    video_script = llm.generate_script(video_subject=video_subject, language=params.video_language,
                                       paragraph_number=paragraph_number)
else:
    logger.debug(f"video script: \n{video_script}")
sm.state.update_task(task_id, state=const.TASK_STATE_PROCESSING, progress=10)

2024-05-13 22:43:48 | INFO | "./../../../../var/folders/20/h5ktr5ls0zgdjw1dtlxcrlpc0000gn/T/ipykernel_23587/2255889761.py:6": <module> - 

## generating video script
2024-05-13 22:43:48 | INFO | "./app/services/llm.py:215": generate_script - subject: 怎么快速赚到100万
2024-05-13 22:43:48 | INFO | "./app/services/llm.py:16": _generate_response - llm provider: openai
2024-05-13 22:43:52 | SUCCESS | "./app/services/llm.py:248": generate_script - completed: 
首先，要明确目标，制定详细的计划。然后，要寻找合适的投资机会，抓住时机进行投资。消除不必要的开支，节约每一分钱。最后，坚持不懈，持续努力，积极应对市场变化。这样，才能快速赚到100万。


In [7]:
logger.info("\n\n## generating video terms")
video_terms = params.video_terms
if not video_terms:
    video_terms = llm.generate_terms(video_subject=video_subject, video_script=video_script, amount=5)
else:
    if isinstance(video_terms, str):
        video_terms = [term.strip() for term in re.split(r'[,，]', video_terms)]
    elif isinstance(video_terms, list):
        video_terms = [term.strip() for term in video_terms]
    else:
        raise ValueError("video_terms must be a string or a list of strings.")
    logger.debug(f"video terms: {utils.to_json(video_terms)}")
script_file = path.join(utils.task_dir(task_id), f"script.json")
script_data = {
    "script": video_script,
    "search_terms": video_terms,
    "params": params,
}
with open(script_file, "w", encoding="utf-8") as f:
    f.write(utils.to_json(script_data))
sm.state.update_task(task_id, state=const.TASK_STATE_PROCESSING, progress=20)

2024-05-13 22:43:52 | INFO | "./../../../../var/folders/20/h5ktr5ls0zgdjw1dtlxcrlpc0000gn/T/ipykernel_23587/447892559.py:1": <module> - 

## generating video terms
2024-05-13 22:43:52 | INFO | "./app/services/llm.py:279": generate_terms - subject: 怎么快速赚到100万
2024-05-13 22:43:52 | INFO | "./app/services/llm.py:16": _generate_response - llm provider: openai
2024-05-13 22:43:53 | SUCCESS | "./app/services/llm.py:300": generate_terms - completed: 
['fast money', 'investment opportunities', 'save money', 'financial goals', 'market changes']


In [8]:
import asyncio
import os
import re
from datetime import datetime
from xml.sax.saxutils import unescape
from edge_tts.submaker import mktimestamp
from loguru import logger
from edge_tts import submaker, SubMaker
import edge_tts
from moviepy.video.tools import subtitles

from app.config import config
from app.utils import utils
import nest_asyncio

nest_asyncio.apply()
def azure_tts_v1(text: str, voice_name: str, voice_file: str) -> [SubMaker, None]:
    text = text.strip()
    for i in range(3):
        # try:
            logger.info(f"start, voice name: {voice_name}, try: {i + 1}")

            async def _do() -> SubMaker:
                communicate = edge_tts.Communicate(text, voice_name)
                sub_maker = edge_tts.SubMaker()
                with open(voice_file, "wb") as file:
                    async for chunk in communicate.stream():
                        if chunk["type"] == "audio":
                            file.write(chunk["data"])
                        elif chunk["type"] == "WordBoundary":
                            sub_maker.create_sub((chunk["offset"], chunk["duration"]), chunk["text"])
                return sub_maker

            # sub_maker = asyncio.get_event_loop().run_until_complete(_do())
            sub_maker = asyncio.run(_do())
            if not sub_maker or not sub_maker.subs:
                logger.warning(f"failed, sub_maker is None or sub_maker.subs is None")
                continue

            logger.info(f"completed, output file: {voice_file}")
            return sub_maker
        # except Exception as e:
        #     logger.error(f"failed, error: {str(e)}")
    return None

In [9]:

audio_file = path.join(utils.task_dir(task_id), f"audio.mp3")
sub_maker = azure_tts_v1(text=video_script, voice_name=voice_name, voice_file=audio_file)
if sub_maker is None:
    sm.state.update_task(task_id, state=const.TASK_STATE_FAILED)
    logger.error(
        """failed to generate audio:
1. check if the language of the voice matches the language of the video script.
2. check if the network is available. If you are in China, it is recommended to use a VPN and enable the global traffic mode.
    """.strip()
    )
    
audio_duration = voice.get_audio_duration(sub_maker)
audio_duration = math.ceil(audio_duration)
sm.state.update_task(task_id, state=const.TASK_STATE_PROCESSING, progress=30)

2024-05-13 22:43:53 | INFO | "./../../../../var/folders/20/h5ktr5ls0zgdjw1dtlxcrlpc0000gn/T/ipykernel_23587/3954587724.py:21": azure_tts_v1 - start, voice name: zh-CN-XiaoxiaoNeural, try: 1
2024-05-13 22:43:56 | INFO | "./../../../../var/folders/20/h5ktr5ls0zgdjw1dtlxcrlpc0000gn/T/ipykernel_23587/3954587724.py:40": azure_tts_v1 - completed, output file: /Users/roper/work/auto_video_generator/storage/tasks/task_id_test_123/audio.mp3


In [10]:
subtitle_path = ""
if params.subtitle_enabled:
    subtitle_path = path.join(utils.task_dir(task_id), f"subtitle.srt")
    subtitle_provider = 'whisper' # config.app.get("subtitle_provider", "").strip().lower()
    logger.info(f"\n\n## generating subtitle, provider: {subtitle_provider}")
    subtitle_fallback = False
    if subtitle_provider == "edge":
        voice.create_subtitle(text=video_script, sub_maker=sub_maker, subtitle_file=subtitle_path)
        if not os.path.exists(subtitle_path):
            subtitle_fallback = True
            logger.warning("subtitle file not found, fallback to whisper")
    if subtitle_provider == "whisper" or subtitle_fallback:
        subtitle.create(audio_file=audio_file, subtitle_file=subtitle_path)
        logger.info("\n\n## correcting subtitle")
        subtitle.correct(subtitle_file=subtitle_path, video_script=video_script)
    subtitle_lines = subtitle.file_to_subtitles(subtitle_path)
    if not subtitle_lines:
        logger.warning(f"subtitle file is invalid: {subtitle_path}")
        subtitle_path = ""

2024-05-13 22:43:56 | INFO | "./../../../../var/folders/20/h5ktr5ls0zgdjw1dtlxcrlpc0000gn/T/ipykernel_23587/1007295417.py:5": <module> - 

## generating subtitle, provider: whisper
2024-05-13 22:43:56 | INFO | "./app/services/subtitle.py:26": create - loading model: base, device: CPU, compute_type: int8
2024-05-13 22:43:57 | INFO | "./app/services/subtitle.py:40": create - start, output file: /Users/roper/work/auto_video_generator/storage/tasks/task_id_test_123/subtitle.srt


Estimating duration from bitrate, this may be inaccurate


2024-05-13 22:43:57 | INFO | "./app/services/subtitle.py:52": create - detected language: 'zh', probability: 1.00
2024-05-13 22:43:58 | DEBUG | "./app/services/subtitle.py:63": recognized - [0.00s -> 0.60s] 首先
2024-05-13 22:43:58 | DEBUG | "./app/services/subtitle.py:63": recognized - [0.94s -> 1.84s] 要明確目標
2024-05-13 22:43:58 | DEBUG | "./app/services/subtitle.py:63": recognized - [2.20s -> 3.48s] 制定詳細的計畫
2024-05-13 22:43:58 | DEBUG | "./app/services/subtitle.py:63": recognized - [4.14s -> 4.54s] 然後
2024-05-13 22:43:58 | DEBUG | "./app/services/subtitle.py:63": recognized - [4.94s -> 6.70s] 要尋找合適的投資機會
2024-05-13 22:43:58 | DEBUG | "./app/services/subtitle.py:63": recognized - [7.14s -> 8.72s] 抓住時機進行投資
2024-05-13 22:43:58 | DEBUG | "./app/services/subtitle.py:63": recognized - [9.34s -> 10.80s] 消除不必要的開支
2024-05-13 22:43:58 | DEBUG | "./app/services/subtitle.py:63": recognized - [11.20s -> 12.26s] 結約每一分錢
2024-05-13 22:43:58 | DEBUG | "./app/services/subtitle.py:63": recognized - [12.96s

In [11]:
import pysrt

In [12]:
subtitle_path

'/Users/roper/work/auto_video_generator/storage/tasks/task_id_test_123/subtitle.srt'

In [13]:
subs = pysrt.open(subtitle_path)

In [14]:
len(subs)

14

In [15]:
dir(subs[0])

['ITEM_PATTERN',
 'TIMESTAMP_SEPARATOR',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_cmpkey',
 '_compare',
 'characters_per_second',
 'duration',
 'end',
 'from_lines',
 'from_string',
 'index',
 'position',
 'shift',
 'split_timestamps',
 'start',
 'text',
 'text_without_tags']

In [16]:
subs[0].text

'首先'

In [17]:
subs[2].duration.seconds

1

In [18]:
import logging
import re
import json
from typing import List
from loguru import logger
from openai import OpenAI
from openai import AzureOpenAI
from openai.types.chat import ChatCompletion

from app.config import config

In [19]:

def generate_picture_term(video_subject: str, subtitle_str: str, amount: int = 5) -> List[str]:
    prompt = f"""
# Role: Picture Search Term Generator

## Goals:
Generate one search term for stock picture, depending on the subject of a video, and one subtite sentence.

## Constrains:
1. the search term are to be returned as a string.
2. search term should consist of 1-3 words, should relate to the subtitle and subtitle str.
3. you must only return one search term. you must not return anything else. you must not return the script.
4. the search term must be related to the subject of the video.
5. reply with english search terms only.

## Output Example:
"search term 1"

## Context:
### Video Subject
{video_subject}

### subtitle_str
{subtitle_str}

Please note that you must use English for generating video search terms; Chinese is not accepted.
""".strip()

    logger.info(f"subject: {video_subject}, subtitle_str:{subtitle_str}")
    # logger.debug(f"prompt: \n{prompt}")
    response = llm._generate_response(prompt)
    search_term = ""

    try:
        search_term = response
        print(search_term)
        if not isinstance(search_term, str) :
            raise ValueError("response is not a list of strings.")

    except (json.JSONDecodeError, ValueError):
        # logger.warning(f"gpt returned an unformatted response. attempting to clean...")
        # Attempt to extract list-like string and convert to list
        return video_subject

    logger.success(f'completed: \n{search_term}')
    return search_term

In [20]:
generate_picture_term(video_subject,  "一打造穩定的現金流是賺取100萬的關鍵")

2024-05-13 22:43:59 | INFO | "./../../../../var/folders/20/h5ktr5ls0zgdjw1dtlxcrlpc0000gn/T/ipykernel_23587/2965486547.py:28": generate_picture_term - subject: 怎么快速赚到100万, subtitle_str:一打造穩定的現金流是賺取100萬的關鍵
2024-05-13 22:43:59 | INFO | "./app/services/llm.py:16": _generate_response - llm provider: openai
"cash flow"
2024-05-13 22:44:03 | SUCCESS | "./../../../../var/folders/20/h5ktr5ls0zgdjw1dtlxcrlpc0000gn/T/ipykernel_23587/2965486547.py:44": generate_picture_term - completed: 
"cash flow"


'"cash flow"'

In [21]:
import os
import random
from urllib.parse import urlencode

import requests
from typing import List
from loguru import logger
from moviepy.video.io.VideoFileClip import VideoFileClip

from app.config import config
from app.models.schema import VideoAspect, VideoConcatMode, MaterialInfo
from app.utils import utils

In [22]:
import random
def search_picture(search_term: str):
    # search_term = "financial stability"
    aspect = VideoAspect(params.video_aspect)
    video_orientation = aspect.name
    video_width, video_height = aspect.to_resolution()
    headers = {
        "Authorization": material.round_robin_api_key()
    }
    proxies = config.pexels.get("proxies", None)
    # Build URL
    url_params = {
        "query": search_term,
        "per_page": 20,
        "orientation": video_orientation
    }
    query_url = f"https://api.pexels.com/v1/search?{urlencode(url_params)}"
    logger.info(f"searching videos: {query_url}, with proxies: {proxies}")
    try:
        r = requests.get(query_url, headers=headers, proxies=proxies, verify=False, timeout=(30, 60))
        response = r.json()
        photos = response.get("photos", [])
        # photos = [photo["url"]for photo in photos if photo.get("url", None)]
        photos = [photo["src"]["original"] for photo in photos if photo.get("src", None)]
        if len(photos) > 0:
            return random.choice(photos)
        return None
    except Exception as e:
        logger.error(f"failed to search videos: {str(e)}")
    
    return None

In [23]:
picture_url = search_picture('financial stability')
print(picture_url)

2024-05-13 22:44:03 | INFO | "./../../../../var/folders/20/h5ktr5ls0zgdjw1dtlxcrlpc0000gn/T/ipykernel_23587/1684953646.py:18": search_picture - searching videos: https://api.pexels.com/v1/search?query=financial+stability&per_page=20&orientation=portrait, with proxies: {}
https://images.pexels.com/photos/3943723/pexels-photo-3943723.jpeg


In [24]:
def get_picture_for_subtitle(video_subject, subtitle_path):
    subs = pysrt.open(subtitle_path)
    picture_urls = []
    for sub in subs:
        subtitle_str = sub.text
        search_term = generate_picture_term(video_subject, subtitle_str)
        pic = search_picture(search_term)
        picture_urls.append(pic)
        logger.info(f'len pics:{len(picture_urls)}, pic:{pic}')
    return picture_urls

In [25]:
all_picture_urls = get_picture_for_subtitle(video_subject, subtitle_path)

2024-05-13 22:44:10 | INFO | "./../../../../var/folders/20/h5ktr5ls0zgdjw1dtlxcrlpc0000gn/T/ipykernel_23587/2965486547.py:28": generate_picture_term - subject: 怎么快速赚到100万, subtitle_str:首先
2024-05-13 22:44:10 | INFO | "./app/services/llm.py:16": _generate_response - llm provider: openai
"money goal"
2024-05-13 22:44:11 | SUCCESS | "./../../../../var/folders/20/h5ktr5ls0zgdjw1dtlxcrlpc0000gn/T/ipykernel_23587/2965486547.py:44": generate_picture_term - completed: 
"money goal"
2024-05-13 22:44:11 | INFO | "./../../../../var/folders/20/h5ktr5ls0zgdjw1dtlxcrlpc0000gn/T/ipykernel_23587/1684953646.py:18": search_picture - searching videos: https://api.pexels.com/v1/search?query=%22money+goal%22&per_page=20&orientation=portrait, with proxies: {}
2024-05-13 22:44:12 | INFO | "./../../../../var/folders/20/h5ktr5ls0zgdjw1dtlxcrlpc0000gn/T/ipykernel_23587/1145771832.py:9": get_picture_for_subtitle - len pics:1, pic:https://images.pexels.com/photos/5632381/pexels-photo-5632381.jpeg
2024-05-13 22:44

In [26]:
all_picture_urls

['https://images.pexels.com/photos/5632381/pexels-photo-5632381.jpeg',
 'https://images.pexels.com/photos/5080677/pexels-photo-5080677.jpeg',
 'https://images.pexels.com/photos/7821484/pexels-photo-7821484.jpeg',
 'https://images.pexels.com/photos/6801872/pexels-photo-6801872.jpeg',
 'https://images.pexels.com/photos/4386154/pexels-photo-4386154.jpeg',
 'https://images.pexels.com/photos/4498143/pexels-photo-4498143.jpeg',
 'https://images.pexels.com/photos/4694569/pexels-photo-4694569.jpeg',
 'https://images.pexels.com/photos/3943734/pexels-photo-3943734.jpeg',
 'https://images.pexels.com/photos/4065620/pexels-photo-4065620.jpeg',
 'https://images.pexels.com/photos/7718482/pexels-photo-7718482.jpeg',
 'https://images.pexels.com/photos/4065897/pexels-photo-4065897.jpeg',
 'https://images.pexels.com/photos/6801872/pexels-photo-6801872.jpeg',
 'https://images.pexels.com/photos/6801869/pexels-photo-6801869.jpeg',
 'https://images.pexels.com/photos/20059159/pexels-photo-20059159.jpeg']

In [27]:
from moviepy.editor import ImageClip, concatenate_videoclips, AudioFileClip
import pysrt
import requests
from tempfile import NamedTemporaryFile
import os

def download_image(url):
    # Download the image and return the path to the temporary file
    response = requests.get(url)
    if response.status_code == 200:
        temp_file = NamedTemporaryFile(delete=False, suffix='.jpg')
        temp_file.write(response.content)
        temp_file.close()
        return temp_file.name
    else:
        raise Exception(f"Failed to download image from {url}")

def make_video(srt_file_path, picture_urls, sound_file_path):
    # Read the subtitle file
    subs = pysrt.open(srt_file_path)
    
    # Create a list of video clips
    clips = []
    temp_files = []
    
    for sub, picture_url in zip(subs, picture_urls):
        # Download the image to a temporary file
        temp_image_path = download_image(picture_url)
        temp_files.append(temp_image_path)
        
        # Calculate the start and end times of the subtitle (in seconds)
        start_time = sub.start.ordinal / 1000  # ordinal is in milliseconds
        end_time = sub.end.ordinal / 1000
        duration = end_time - start_time
        
        # Create an image clip with the specified duration
        img_clip = ImageClip(temp_image_path, duration=duration).set_fps(24)
        
        # Set the start time of the clip
        img_clip = img_clip.set_start(start_time)
        
        # Add the clip to the list
        clips.append(img_clip)
    
    # Concatenate all video clips with an explicit fps set
    video = concatenate_videoclips(clips, method="compose").set_fps(24)
    
    # Load the audio file
    audio = AudioFileClip(sound_file_path)
    
    # Set the audio of the concatenated video clip
    final_video = video.set_audio(audio)
    
    # Set the duration of the final video to match the audio duration
    final_video = final_video.set_duration(audio.duration)
    
    # Output the final video file
    final_video.write_videofile("output_video.mp4", codec="libx264", fps=24)
    
    # Clean up temporary files
    for temp_file in temp_files:
        os.unlink(temp_file)


FileNotFoundError: [Errno 2] No such file or directory: 'example.srt'

In [ ]:
make_video(subtitle_path, all_picture_urls, audio_file)

Moviepy - Building video output_video.mp4.
MoviePy - Writing audio in output_videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video output_video.mp4



TypeError: must be real number, not NoneType

In [28]:
srt_file_path = subtitle_path
picture_urls = all_picture_urls
sound_file_path = audio_file 
print(f'srt_file_path:{srt_file_path}, picture_urls:{picture_urls}, sound_file_path:{sound_file_path}')
    
# 读取字幕文件
subs = pysrt.open(srt_file_path)

# 创建视频片段列表
temp_files = []

for picture_url in picture_urls:
    print(f'start handle picture_url:{picture_url}')
    # 下载图片到临时文件
    temp_image_path = download_image(picture_url)
    temp_files.append(temp_image_path)

print("temp_files len:", len(temp_files))



srt_file_path:/Users/roper/work/auto_video_generator/storage/tasks/task_id_test_123/subtitle.srt, picture_urls:['https://images.pexels.com/photos/5632381/pexels-photo-5632381.jpeg', 'https://images.pexels.com/photos/5080677/pexels-photo-5080677.jpeg', 'https://images.pexels.com/photos/7821484/pexels-photo-7821484.jpeg', 'https://images.pexels.com/photos/6801872/pexels-photo-6801872.jpeg', 'https://images.pexels.com/photos/4386154/pexels-photo-4386154.jpeg', 'https://images.pexels.com/photos/4498143/pexels-photo-4498143.jpeg', 'https://images.pexels.com/photos/4694569/pexels-photo-4694569.jpeg', 'https://images.pexels.com/photos/3943734/pexels-photo-3943734.jpeg', 'https://images.pexels.com/photos/4065620/pexels-photo-4065620.jpeg', 'https://images.pexels.com/photos/7718482/pexels-photo-7718482.jpeg', 'https://images.pexels.com/photos/4065897/pexels-photo-4065897.jpeg', 'https://images.pexels.com/photos/6801872/pexels-photo-6801872.jpeg', 'https://images.pexels.com/photos/6801869/pexels

In [ ]:
len(subs)

14

In [36]:
temp_files

['/var/folders/20/h5ktr5ls0zgdjw1dtlxcrlpc0000gn/T/tmpo0wr901h.jpg',
 '/var/folders/20/h5ktr5ls0zgdjw1dtlxcrlpc0000gn/T/tmp11iyujh7.jpg',
 '/var/folders/20/h5ktr5ls0zgdjw1dtlxcrlpc0000gn/T/tmpzg6jus_p.jpg',
 '/var/folders/20/h5ktr5ls0zgdjw1dtlxcrlpc0000gn/T/tmpjsdzvnm2.jpg',
 '/var/folders/20/h5ktr5ls0zgdjw1dtlxcrlpc0000gn/T/tmpt8o3xo1x.jpg',
 '/var/folders/20/h5ktr5ls0zgdjw1dtlxcrlpc0000gn/T/tmpr_r70tv3.jpg',
 '/var/folders/20/h5ktr5ls0zgdjw1dtlxcrlpc0000gn/T/tmp3riwsjsp.jpg',
 '/var/folders/20/h5ktr5ls0zgdjw1dtlxcrlpc0000gn/T/tmp_vuw60mo.jpg',
 '/var/folders/20/h5ktr5ls0zgdjw1dtlxcrlpc0000gn/T/tmpqfq4zmo4.jpg',
 '/var/folders/20/h5ktr5ls0zgdjw1dtlxcrlpc0000gn/T/tmp8mefqjnn.jpg',
 '/var/folders/20/h5ktr5ls0zgdjw1dtlxcrlpc0000gn/T/tmpz80d_d38.jpg',
 '/var/folders/20/h5ktr5ls0zgdjw1dtlxcrlpc0000gn/T/tmppzykwczt.jpg',
 '/var/folders/20/h5ktr5ls0zgdjw1dtlxcrlpc0000gn/T/tmpad34y5t4.jpg',
 '/var/folders/20/h5ktr5ls0zgdjw1dtlxcrlpc0000gn/T/tmpv40y8fhi.jpg']

In [33]:
clips = []
i = 0
for sub in subs:
    dur = sub.duration.seconds
    if dur <= 0:
        continue
    
    picture = temp_files[i]
    if i < len(temp_files) - 1:
        i += 1
    print(f'start handle sub, duration:{dur} picture:{picture}')
    if False == os.path.exists(picture):
        print(f'picture not exists:{picture}')
        continue
    
    # 创建一个图片剪辑
    img_clip = ImageClip(picture).set_duration(dur)
    
    # 将剪辑添加到列表中
    clips.append(img_clip)

start handle sub, duration:1 picture:/var/folders/20/h5ktr5ls0zgdjw1dtlxcrlpc0000gn/T/tmpo0wr901h.jpg
start handle sub, duration:1 picture:/var/folders/20/h5ktr5ls0zgdjw1dtlxcrlpc0000gn/T/tmp11iyujh7.jpg
start handle sub, duration:1 picture:/var/folders/20/h5ktr5ls0zgdjw1dtlxcrlpc0000gn/T/tmpzg6jus_p.jpg
start handle sub, duration:1 picture:/var/folders/20/h5ktr5ls0zgdjw1dtlxcrlpc0000gn/T/tmpjsdzvnm2.jpg
start handle sub, duration:1 picture:/var/folders/20/h5ktr5ls0zgdjw1dtlxcrlpc0000gn/T/tmpt8o3xo1x.jpg
start handle sub, duration:1 picture:/var/folders/20/h5ktr5ls0zgdjw1dtlxcrlpc0000gn/T/tmpr_r70tv3.jpg
start handle sub, duration:1 picture:/var/folders/20/h5ktr5ls0zgdjw1dtlxcrlpc0000gn/T/tmp3riwsjsp.jpg


In [34]:
# 合并所有视频片段
video = concatenate_videoclips(clips, method="compose")



In [35]:
video.write_videofile("test.mp4", fps=24)

Moviepy - Building video test.mp4.
Moviepy - Writing video test.mp4

ffmpeg binary: ffmpeg (4480, 6720) None libx264 medium None -1 None None None rgb24 


TypeError: must be real number, not NoneType

In [ ]:
# 添加音频
audio = AudioFileClip(sound_file_path)
final_video = video.set_audio(audio)



In [ ]:
# 输出最终视频文件
final_video.write_videofile("output_video.mp4", codec="libx264", fps=24)



Moviepy - Building video output_video.mp4.
MoviePy - Writing audio in output_videoTEMP_MPY_wvf_snd.mp3


chunk:   0%|          | 0/493 [00:00<?, ?it/s, now=None]

MoviePy - Done.
Moviepy - Writing video output_video.mp4



TypeError: must be real number, not NoneType

In [ ]:
# 清理临时文件
for temp_file in temp_files:
    os.unlink(temp_file)